In [5]:
import pandas as pd
import torch
from model.FieldingModel import FieldingModel


In [ ]:
league =  'BZ'
season = 2017
positions = [9,8,7,6,5,4,3,2]
ratings_type = "Standard"

In [7]:

    
for position in positions:
    print(f"\r\nTraining model for position {position}")
    fieldingModel = FieldingModel(league, season, season, position, ratings_type)
    fieldingModel.load_data(position)

    epoch_count, loss = fieldingModel.train(50000)
    print(f"Epoch [{epoch_count}/{50000}], Loss: {loss:.7f}")

    test_loss = fieldingModel.evaluate()
    print(f"Test Mean Squared Error: {test_loss:.7f}")

    feature_importance = fieldingModel.feature_importance()
    print(feature_importance)

    fieldingModel.save_model()


Training model for position 9
CUDA is available! Using GPU.
Epoch [356/50000], Loss: 0.0009264
Epoch [355/50000], Loss: 0.0009264
Test Mean Squared Error: 1037.9387207
   mean_abs_shap  stdev_abs_shap        name
3      16.278492       13.082121  OFRngDelta
4      13.540955       14.868732  OFArmDelta
7      11.475166        9.858417          ID
1       7.996900        8.256564          WT
0       6.744585        5.705916         Age
6       4.626740        4.022025          RF
2       4.581750        5.249155         SPE
5       4.052957        3.984397  OFERRDelta

Training model for position 8
CUDA is available! Using GPU.
Epoch [874/50000], Loss: 0.0009110
Epoch [873/50000], Loss: 0.0009110
Test Mean Squared Error: 470.8274536
   mean_abs_shap  stdev_abs_shap        name
6      10.348074        9.504529          CF
2       7.853105        5.345257         SPE
3       5.249881        3.521076  OFRngDelta
1       5.051462        4.088517          WT
5       3.912333        3.923519 